In [ ]:
pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
tor

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
import evaluate
import torch
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    ModernBertForSequenceClassification
)
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {0: "negative", 1: "neutral", 2: "positive"}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Newton/SemEval-Restaurants/data/train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/Newton/SemEval-Restaurants/data/val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Newton/SemEval-Restaurants/data/test.csv")

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")

def tokenize_function(example):
    combined_input = f"aspect: {example['aspect']} text: {example['text']}"

    encoding = tokenizer(
        combined_input,
        truncation=True,
        padding="max_length",
        max_length=256
    )
    # Convert label from string to int
    encoding["label"] = label2id[example["label"]]
    return encoding

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=False)
val_dataset = val_dataset.map(tokenize_function, batched=False)
test_dataset = test_dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/2521 [00:00<?, ? examples/s]

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

Map:   0%|          | 0/361 [00:00<?, ? examples/s]

In [ ]:
model = ModernBertForSequenceClassification.from_pretrained(
    "answerdotai/ModernBERT-base",
    num_labels=3
)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    logging_dir="./logs",
    logging_steps=200,
    warmup_ratio=0.06,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fialavi2 (fialavi2-czech-technical-university-in-prague) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W0404 10:12:57.723000 557 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.549413,0.763889
2,0.728300,0.439178,0.829167
3,0.366900,0.536704,0.854167
4,0.181700,0.854877,0.854167
5,0.181700,0.985141,0.863889
6,0.072100,1.011584,0.862500
7,0.022400,1.273275,0.863889
8,0.007100,1.257022,0.865278
9,0.008900,1.237130,0.869444
10,0.008900,1.244268,0.866667


TrainOutput(global_step=1580, training_loss=0.176147974518281, metrics={'train_runtime': 1768.5266, 'train_samples_per_second': 14.255, 'train_steps_per_second': 0.893, 'total_flos': 4295282426680320.0, 'train_loss': 0.176147974518281, 'epoch': 10.0})

In [ ]:
val_results = trainer.evaluate()
print(val_results)

{'eval_loss': 1.237130045890808, 'eval_accuracy': 0.8694444444444445, 'eval_runtime': 14.2429, 'eval_samples_per_second': 50.552, 'eval_steps_per_second': 3.159, 'epoch': 10.0}


In [ ]:
test_results = trainer.evaluate(test_dataset)
print(test_results)

{'eval_loss': 1.332175374031067, 'eval_accuracy': 0.8365650969529086, 'eval_runtime': 7.6227, 'eval_samples_per_second': 47.359, 'eval_steps_per_second': 3.017, 'epoch': 10.0}


In [ ]:
trainer.save_model("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/ModernBERT-base")

In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/ModernBERT-base")

('/content/drive/MyDrive/Newton/SemEval-Restaurants/models/ModernBERT-base/tokenizer_config.json',
 '/content/drive/MyDrive/Newton/SemEval-Restaurants/models/ModernBERT-base/special_tokens_map.json',
 '/content/drive/MyDrive/Newton/SemEval-Restaurants/models/ModernBERT-base/tokenizer.json')

In [ ]:
model = ModernBertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/ModernBERT-base")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/ModernBERT-base")

## Vyhodnocovací metriky

In [ ]:
def compute_metrics_accuracy(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer_accuracy = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_accuracy
)

In [ ]:
test_results_accuracy = trainer_accuracy.evaluate()
print(test_results_accuracy)

{'eval_loss': 1.3321752548217773, 'eval_model_preparation_time': 0.0032, 'eval_accuracy': 0.8365650969529086, 'eval_runtime': 7.811, 'eval_samples_per_second': 46.217, 'eval_steps_per_second': 5.889}


In [ ]:
def compute_metrics_weightedf1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

trainer_weightedf1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_weightedf1
)

In [ ]:
test_results_weightedf1 = trainer_weightedf1.evaluate()
print(test_results_weightedf1)

{'eval_loss': 1.3321752548217773, 'eval_model_preparation_time': 0.0029, 'eval_f1': 0.8371969384952064, 'eval_runtime': 7.8321, 'eval_samples_per_second': 46.092, 'eval_steps_per_second': 5.873}


In [ ]:
def compute_metrics_macrof1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="macro")

trainer_macrof1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_macrof1
)

In [ ]:
test_results_macrof1 = trainer_macrof1.evaluate()
print(test_results_macrof1)

{'eval_loss': 1.3321752548217773, 'eval_model_preparation_time': 0.0034, 'eval_f1': 0.783768039189726, 'eval_runtime': 8.1062, 'eval_samples_per_second': 44.534, 'eval_steps_per_second': 5.675}


## Praktické testy

In [ ]:
def predict_sentiment(model, tokenizer, aspect, text):
    combined_input = f"aspect: {aspect} text: {text}"

    encoding = tokenizer(
        text=combined_input,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=256
    )
    with torch.no_grad():
        outputs = model(**encoding)

    logits = outputs.logits
    print(logits)
    predicted_class_id = logits.argmax(dim=1).item()
    return id2label[predicted_class_id]

### Sada L

In [ ]:
example_aspect = "power"
example_text = "The power is good, but the display is blurry and dark. The battery was a real surprise, I don't need to charge it during the day!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L1: {prediction}, expected: positive")
example_aspect = "display"
example_text = "The power is good, but the display is blurry and dark. The battery was a real surprise, I don't need to charge it during the day!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L2: {prediction}, expected: negative")
example_aspect = "battery"
example_text = "The power is good, but the display is blurry and dark. The battery was a real surprise, I don't need to charge it during the day!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L3: {prediction}, expected: positive")

tensor([[-2.1429, -5.6322,  7.0335]])
Predicted sentiment for L1: positive, expected: positive
tensor([[ 9.0907, -8.2138, -3.4124]])
Predicted sentiment for L2: negative, expected: negative
tensor([[ 0.4293, -6.7057,  6.4813]])
Predicted sentiment for L3: positive, expected: positive


### Sada R

In [ ]:
example_aspect = "staff"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R1: {prediction}, expected: negative")
example_aspect = "table"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R2: {prediction}, expected: negative")
example_aspect = "food"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R3: {prediction}, expected: positive")
example_aspect = "beer"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R4: {prediction}, expected: positive")

tensor([[ 6.5475, -6.0880, -4.0760]])
Predicted sentiment for R1: negative, expected: negative
tensor([[ 6.8077, -4.5272, -7.6359]])
Predicted sentiment for R2: negative, expected: negative
tensor([[-4.2025, -4.6476,  6.2122]])
Predicted sentiment for R3: positive, expected: positive
tensor([[-4.1125, -4.9716,  6.0713]])
Predicted sentiment for R4: positive, expected: positive


### Sada M

In [ ]:
example_aspect = "Osaka"
example_text = "At EXPO 2025 in Osaka, many companies introduced their latest products. Saremi successfully introduced a new laptop that visitors loved, whereas Babtel's new phone was a disaster, it didn't even work!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M1: {prediction}, expected: neutral")
example_aspect = "Saremi"
example_text = "At EXPO 2025 in Osaka, many companies introduced their latest products. Saremi successfully introduced a new laptop that visitors loved, whereas Babtel's new phone was a disaster, it didn't even work!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M2: {prediction}, expected: positive")
example_aspect = "Babtel"
example_text = "At EXPO 2025 in Osaka, many companies introduced their latest products. Saremi successfully introduced a new laptop that visitors loved, whereas Babtel's new phone was a disaster, it didn't even work!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M3: {prediction}, expected: negative")

tensor([[-5.4994,  1.7525, -1.6156]])
Predicted sentiment for M1: neutral, expected: neutral
tensor([[-4.1143, -4.8040,  7.1822]])
Predicted sentiment for M2: positive, expected: positive
tensor([[ 6.9015, -6.1587, -3.2582]])
Predicted sentiment for M3: negative, expected: negative


### Sada O

In [ ]:
example_aspect = "day"
example_text = "Today was a really good day, Anna asked me out for a date! Unfortunately the weather was awful, so we had to cancel it."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O1: {prediction}, expected: positive")
example_aspect = "weather"
example_text = "Today was a really good day, Anna asked me out for a date! Unfortunately the weather was awful, so we had to cancel it."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O2: {prediction}, expected: negative")

tensor([[-3.8389, -5.3983,  6.9026]])
Predicted sentiment for O1: positive, expected: positive
tensor([[ 7.0498, -7.7010, -1.3662]])
Predicted sentiment for O2: negative, expected: negative
